In [1]:
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
from scipy.stats import shapiro
from scipy.stats import kstest
from scipy import stats as s
import statistics
import datetime
from configparser import ConfigParser


In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
# CODE FOR INSERTION IN AUDIT INTEGRATION 
cursor.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'REPORT_RISK',GETDATE(),'Error','RISK_REPORT','RISK_TRANSACTION_SUMMARY_STG',(SELECT MAX(T.MAX_DATE) FROM (select MAX(CREATE_DATE) AS MAX_DATE from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG UNION ALL select MAX(UPDATE_DATE) AS MAX_DATE from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG )T))""")

job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN with(nolock) where job_name = 'REPORT_RISK' ",conn)
job_id = job_id.iloc[0,0]
rules_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with(nolock)", conn)
conn.commit()


In [4]:
cursor.execute('''Delete from [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] ''')
conn.commit()

In [5]:
# ALL THE DATA WILL BE CONSIDERED IN CASE OF THRESHOLD CALCULATION AND REPORT RISK TO BE RECALCULATED.
cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] 
(
 REPORT_ID
 )
SELECT  DISTINCT REPORT_ID FROM [FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_SUMMARY_STG with (nolock)''')
conn.commit()

In [6]:
# CALCULATING THE NUMBER OF RECOREDS TO BE CALCULATED.
no_of_records = pd.read_sql_query("SELECT COUNT(REPORT_ID) FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_INCREMENTAL WITH (NOLOCK)",conn)
no_of_records = no_of_records.iloc[0,0] 

In [7]:
# STORING VARIABLES USED TO UPDATE AUDIT INTGN.
%store job_id
%store no_of_records

Stored 'job_id' (int64)
Stored 'no_of_records' (int64)


In [8]:
cursor.execute('''
insert into FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG 
select DISTINCT BATCH_ID,REPORT_ID,YEAR_REPORT,MONTH_REPORT,REPORT_TYPE,GETDATE() AS CREATE_DATE,NULL AS UPDATE_DATE,NULL AS TRAN_AMNT_CREDIT,NULL AS TRAN_AMNT_DEBIT from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG
''')
conn.commit()

In [9]:
# UPDATE DEBIT AMOUNT OF A REPORT
tran_amnt_debit = '''UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG  SET TRAN_AMNT_DEBIT  = A.TRAN_AMNT_DEBIT
FROM (
SELECT  A.REPORT_ID,SUM(B.TRAN_AMNT) AS TRAN_AMNT_DEBIT
FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG A
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG_TRAN B
ON A.REPORT_ID = B.REPORT_ID
WHERE B.CR_DR_DEP_WITH IN ('DEBIT','Withdrawal','Buy','ATM Cash Withdrawal')
GROUP BY A.REPORT_ID
)a
WHERE a.REPORT_ID = FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG.REPORT_ID'''

In [10]:
# UPDATE CREDIT AMOUNT OF A REPORT
tran_amnt_credit = '''
UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG  SET TRAN_AMNT_CREDIT  = A.TRAN_AMNT_CREDIT
FROM (
SELECT  A.REPORT_ID,SUM(B.TRAN_AMNT) AS TRAN_AMNT_CREDIT
FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG A
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG_TRAN B
ON A.REPORT_ID = B.REPORT_ID
WHERE (B.CR_DR_DEP_WITH IN ('Credit','Deposit','Sell') or CR_DR_DEP_WITH  IS NULL)
GROUP BY A.REPORT_ID
)a
WHERE a.REPORT_ID = FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG.REPORT_ID 
'''

In [11]:
cursor.execute(tran_amnt_credit)
cursor.execute(tran_amnt_debit)
conn.commit()

In [12]:
rules_df = pd.read_sql_query('''SELECT * FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES WITH (NOLOCK)''',conn)

In [13]:
rules_df = rules_df[rules_df['PERSON_TYPE'] == 'Report']

In [14]:
rules_df

RULE_ID     RULE_NO                                          RULE_DESC  \
5      5.25  R005025000  Donation from a single entity is greater than ...   
6      5.26  R005026000  High value of transactions received from one o...   
154    5.21  R005021000                  Cancellation of FCRA Registration   
155    5.22  R005022000                    Suspension of FCRA Registration   
156    5.23  R005023000                                   Black listed NGO   
157    5.24  R005024000                 Existence of FCRA account with NPO   
169    5.18  R005018000  High value funds  sent from the same person to...   
186    5.19  R005019000  High value funds received from the same person...   
187     5.2  R005002000  Untraceable flags identified (E.g. true accoun...   
188    5.20  R005020000  High number of transactions to newly created o...   
197     5.1  R005001000  Any of reported parties already have trigerred...   
198    5.10  R005010000  High value cross border fund transfers are sen...   
199    5.11  R005011000  High value of transactions transfer out to sen...   
200    5.12  R005012000  High value of transactions transfer in to bene...   
201    5.14  R005014000                       Source of Funds is "Unknown"   
202    5.15  R005015000                  Destination of Funds is "Unknown"   
203    5.16  R005016000              Identity of the remitter is "Unknown"   
204    5.17  R005017000  <count of persons> entities/individuals as men...   
205    5.27  R005027000  Location of detection of counterfeit instrumen...   
206    5.28  R005028000  Notional/face value of the counterfeit currenc...   
207    5.29  R005029000  Deonomination Value <value> and Number of piec...   
208     5.3  R005003000  GoS tags reveals  high risk tags on any of ent...   
209    5.30  R005030000  <count of high risk report> of high risk trans...   
210     5.4  R005004000  High Risk Purpose code reported in CBWTR: <pur...   
211     5.5  R005005000  Value of transaction(s) reported is greater th...   
212     5.6  R005006000  Number of transaction(s) reported is greater t...   
213     5.7  R005007000  Number of involved entities/individuals report...   
214     5.8  R005008000  Number of accounts involved are greater than p...   
215     5.9  R005009000  Number of linked accounts are greater than par...   

    PERSON_TYPE         RISK_TYPE  IS_ACTIVE_FLAG CREATE_DATE UPDATE_DATE  \
5        Report       Report Risk               0  2022-12-07  1900-01-01   
6        Report       Report Risk               0  2022-12-07  1900-01-01   
154      Report       Report Risk               0  2022-12-07  1900-01-01   
155      Report       Report Risk               0  2022-12-07  1900-01-01   
156      Report       Report Risk               0  2022-12-07  1900-01-01   
157      Report       Report Risk               0  2022-12-07  1900-01-01   
169      Report       Report Risk               0  2022-12-07  1900-01-01   
186      Report       Report Risk               0  2022-12-07  1900-01-01   
187      Report   Auto High Rules               0  2022-12-07  1900-01-01   
188      Report       Report Risk               0  2022-12-07  1900-01-01   
197      Report  Auto High Rules                1  2022-12-07  1900-01-01   
198      Report       Report Risk               0  2022-12-07  1900-01-01   
199      Report       Report Risk               0  2022-12-07  1900-01-01   
200      Report       Report Risk               0  2022-12-07  1900-01-01   
201      Report       Report Risk               1  2022-12-07  1900-01-01   
202      Report       Report Risk               1  2022-12-07  1900-01-01   
203      Report       Report Risk               0  2022-12-07  1900-01-01   
204      Report       Report Risk               1  2022-12-07  1900-01-01   
205      Report       Report Risk               1  2022-12-07  1900-01-01   
206      Report       Report Risk               1  2022-12-07  1900-01-01   
207      Report       Report Risk             

In [15]:
# RUNNING REPORT RISK QUERIES TO POPULATE RISK_REPORT_RULES.

for query in rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) ]['Query'].to_list():
    pprint(query)
    cursor.execute(query)
    print("Done")
#     print(namestr(query,globals())[0]," DONE!")
    conn.commit()
    


('MERGE INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_RULES WITH (HOLDLOCK) AS '
 'TARGET USING           (   \r\n'
 'SELECT          DISTINCT RTSSE.BATCH_ID,    \r\n'
 'RTSSE.REPORT_ID,      \r\n'
 "'R005001000' AS RULE_NO,  \r\n"
 'RPSS.ORGANIZATION_MASTER_ID AS ENTITY_ID ,    \r\n'
 '10 AS RISK_SCORE_STATIC,   \r\n'
 'GETDATE() AS CREATE_DATE   \r\n'
 'FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG_ENTITY RTSSE '
 'WITH (NOLOCK)   \r\n'
 'INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES '
 'RPSS  WITH (NOLOCK)  \r\n'
 'ON RTSSE.ENTITY_MASTER_ID=RPSS.ORGANIZATION_MASTER_ID \r\n'
 "WHERE   RTSSE.ENTITY_TYPE = 'O'     AND RPSS.RULE_NO IN (\r\n"
 'SELECT RULE_NO FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES  WITH '
 "(NOLOCK)  WHERE      RISK_TYPE='AUTO HIGH RULES ')    \r\n"
 'UNION \r\n'
 'SELECT    DISTINCT            RTSSE.BATCH_ID,       \r\n'
 "RTSSE.REPORT_ID,             'R005001000' AS RULE_NO,      \r\n"
 'RPSS.PERSON_MASTER_ID AS 

Done
('MERGE INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_RULES WITH '
 '(HOLDLOCK)     AS TARGET USING(       \r\n'
 "   SELECT   RTSST.BATCH_ID,  RTSST.REPORT_ID,  'R005005000' AS RULE_NO,    "
 'COALESCE(SUM(TRAN_AMNT),0) AS RULE_VALUE,   \r\n'
 '   NULL AS RISK_SCORE_STATIC,     GETDATE() AS CREATE_DATE      FROM    '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG_TRAN  RTSST    '
 'WITH (NOLOCK) \r\n'
 '   GROUP BY BATCH_ID,REPORT_ID)    AS SOURCE    ON TARGET.BATCH_ID '
 '=SOURCE.BATCH_ID AND TARGET.REPORT_ID=SOURCE.REPORT_ID AND     \r\n'
 "   TARGET.RULE_NO = 'R005005000'         AND "
 'TARGET.RULE_VALUE=SOURCE.RULE_VALUE WHEN MATCHED THEN UPDATE SET  \r\n'
 '   '
 'TARGET.RULE_VALUE=SOURCE.RULE_VALUE,TARGET.RISK_SCORE_STATIC=SOURCE.RISK_SCORE_STATIC      '
 'WHEN NOT MATCHED BY TARGET THEN  \r\n'
 '   '
 'INSERT(BATCH_ID,REPORT_ID,RULE_NO,RULE_VALUE,RISK_SCORE_STATIC,CREATE_DATE)   \r\n'
 '   VALUES(SOURCE.BATCH_ID,SOURCE.REPORT_ID, '
 'SOURCE.RULE_NO,SOURCE.RUL

In [16]:

risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES]", conn)
static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Report') ]['RULE_NO'].to_list()
auto_high_rules =  risk_lkp[(risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['RISK_TYPE'] == 'Auto High Rules ') & (risk_lkp['PERSON_TYPE'] == 'Report')]['RULE_NO'].to_list()
non_stat_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 0) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Report')]['RULE_NO'].to_list()



In [17]:
auto_high_rules

['R005001000']

In [18]:
# Read RISK_REPORT_RULES for the static rules
if len(auto_high_rules) == 1:
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in ('{}')".format(auto_high_rules[0]),conn)
    
elif(len(auto_high_rules)== 0):
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in ('')",conn)
    
else:
    auto_high_rules = tuple(auto_high_rules)
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in" + str (auto_high_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [19]:
report_rules_auto_high = report_rules['REPORT_ID'].to_list()

In [20]:
report_rules_auto_high

[]

In [21]:
# for report in report_rules_auto_high:
#     print('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
#   select distinct  A.BATCH_ID,A.REPORT_ID,C.REPORT_TYPE,10 as RISK_SCORE,
#   GETDATE () AS CREATE_DATE, NULL AS RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
# from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
# inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
# on C.REPORT_ID = A.REPORT_ID where C.REPORT_ID = {}  '''.format(report_rules_auto_high) )
#     conn.commit()
# #     conn.close()

In [22]:
# INSERT DATA IN RISK_REPORT IF REPORT IS APPLICABLE IN A AUTO HIGH RULE

if len(report_rules_auto_high) == 1:
    
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,10 as RISK_SCORE,
GETDATE () AS CREATE_DATE,
NULL as RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
INNER join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
INNER join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID = {}
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high[0]))
    conn.commit()
    
    
    
elif len(report_rules_auto_high) > 1 :
    print("Populate risk_report for more than 1")
    report_rules_auto_high = tuple(report_rules_auto_high)
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,10 AS RISK_SCORE,
GETDATE() AS CREATE_DATE,
NULL as RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
INNER join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
INNER join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID  IN {} 
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE'''.format(report_rules_auto_high))
    conn.commit()
    

In [23]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [24]:
# non_stat_rules = tuple(non_stat_rules)
stat_rules = tuple(static_rules)

In [25]:
# Read RISK_REPORT_RULES for the static rules
if len(non_stat_rules) == 1:
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in ('{}')".format(non_stat_rules[0]),conn)
    
elif(len(non_stat_rules)== 0):
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in ('')",conn)
    
else:
    non_stat_rules = tuple(non_stat_rules)
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in" + str (non_stat_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [26]:
# REMOVING THE REPORTS THAT FALLS IN AUTO HIGH RULES
report_rules = report_rules[~report_rules['REPORT_ID'].isin(report_rules_auto_high)]

In [27]:
summary_stg = pd.read_sql_query('''select * from fincore_db.fincore_analytics.risk_report_summary_stg''',conn)

In [28]:
summary_stg = summary_stg[['REPORT_ID','REPORT_TYPE']]

In [29]:
peer_groups = pd.read_sql_query('''select distinct report_type from fincore_db.fincore_analytics.risk_report_summary_stg''',conn)

In [30]:
non_stat_rules

('R005001000',
 'R005028000',
 'R005029000',
 'R005005000',
 'R005006000',
 'R005007000',
 'R005008000')

In [31]:
report_rules = report_rules.merge(summary_stg,on = 'REPORT_ID',how = 'left')

In [32]:
# USED FOR THOSE REPORTS WHERE REPORT TYPE IS NULL.
report_rules['REPORT_TYPE'].fillna('UNKNOWN',inplace = True)

In [33]:
peer_groups = report_rules['REPORT_TYPE'].unique()

In [34]:
report_rules['RULE_VALUE'] = report_rules['RULE_VALUE'].astype('float64')

In [35]:
report_rules['RULE_VALUE'] = report_rules['RULE_VALUE'].astype('int64')

In [36]:
report_rules.drop_duplicates(inplace = True)

# THRESHOLD CALCULATION

In [37]:
def not_normal_outliers_removal(d):
    print("not_normal_outliers_removal")
    data = [*set(d)]
    return compute_IQR(d,data)

In [38]:
def compute_IQR(d,data):
    print("compute_IQR")
    q3 = np.quantile(data, 0.75)
    q1 = np.quantile(data, 0.25)
    iqr = ((q3-q1)* 1.5)
    outliers_left = q1 - iqr
    outliers_right = q3 + iqr
    print("outliers_right",outliers_right)
#     data = list(filter(lambda x: outliers_left <= x <= outliers_right, data))
    data = list(filter(lambda x: x <= outliers_right,d))

    return data,outliers_right


In [39]:
def normal_outliers_removal(d):
    print("normal_outliers_removal")
    mean = np.mean(d)
    std = np.std(d)
    outliers_right = (mean + (3*std))
    data = list(filter(lambda x: x <= outliers_right, d))
    print(outliers_right)
    return data,outliers_right

In [40]:
def thresh_cal(d,outliers_right):
    print("thresh")
    median = np.median(d)
    mean = np.mean(d)
    std = np.std(d)
    mode = int(s.mode(d)[0])


    
    if std >= 1:
        one = round(median+std)
        two = round(median+2*std)
        three = round(median+3*std)
        four = round(median+4*std)
        five = round(median+5*std)
        six = round(median+6*std)
        seven = round(median+7*std)
        eight = round(median+8*std)
        nine =round(median+9*std)
        ten = round(median+10*std)

    elif std > 0:
        one = np.ceil(median+std)
        two = np.ceil(one+std)
        three = np.ceil(two+std)
        four = np.ceil(three+std)
        five = np.ceil(four+std)
        six = np.ceil(five+std)
        seven = np.ceil(six+std)
        eight = np.ceil(seven+std)
        nine =np.ceil(eight+std)
        ten = np.ceil(nine+std)
        
        
    elif std == 0:
        one = np.ceil(median+std+1)
        two = np.ceil(one+std+1)
        three = np.ceil(two+std+1)
        four = np.ceil(three+std+1)
        five = np.ceil(four+std+1)
        six = np.ceil(five+std+1)
        seven = np.ceil(six+std+1)
        eight = np.ceil(seven+std+1)
        nine =np.ceil(eight+std+1)
        ten = np.ceil(nine+std+1)

        
    print(one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right)

    return(one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right)

In [41]:

def compute_outlier(d):
    try:
    
        if (len(d) == 0):
            print("Not applicable")
            return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None) 

          

        elif ( len(d) < 5):
            print("Not applicable")
            return thresh_cal(d,"NA") 
            
        
        
        
        else:
            
            if ( len(d) <= 150):
                stats,p_val = shapiro(d)
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)   
                    
            else:
                stats,p_val  = kstest(d, 'norm')
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                    
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
    
            
    except KeyError:
        return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None) 
    
    return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None) 





In [42]:

peer_th = []
for peer_group in  peer_groups:
    temp = report_rules[report_rules['REPORT_TYPE'] == peer_group]
    for rule_no in non_stat_rules:
        data = temp[temp['RULE_NO'] == rule_no]
        data = data['RULE_VALUE'].to_list()
        data = list(map(int,data))

        one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right = compute_outlier(data)
        
        l =[peer_group,rule_no,one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right]
        peer_th.append(l)


Not applicable
Not applicable
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 1025241105.0
thresh
129174487 256848973 384523460 512197947 639872434 767546920 895221407 1022895894 1150570380 1278244867 43559473.955587395 1500000.0 1500000 127674486.71396393 1025241105.0
not_normal_outliers_removal
compute_IQR
outliers_right 10.0
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.3208489388264668 1.0 1 0.9025845969816965 10.0
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 2453419715.0
thresh
524501555 980141218 1435780882 1891420546 2347060209 2802699873 3258339537 3713979200 4169618864 4625258528 456549025.1395349 68861891.0 68861891 455639663.6694273 2453419715.0
not_normal_outliers_removal
compute_IQR
outliers_right 4.0
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.8863636363636365 1.0 1 0.9820180772940136 4.0
Not applicable
Not applicable


In [43]:
# # CALCULATING THE THRESHOLD

# peer_th = []
# for peer_group in  peer_groups:
#     temp = report_rules[report_rules['REPORT_TYPE'] == peer_group]
#     for rule_no in non_stat_rules:
#         data = temp[temp['RULE_NO'] == rule_no]
#         data = data['RULE_VALUE'].to_list()
#         data = list(map(int,data))
#         median = np.median(data)
#         mean = np.mean(data)
#         std = np.std(data)
#         print(std)
#         if std >= 1:
#             print("std>1")
#             one = round(median+std)
#             two = round(median+2*std)
#             three = round(median+3*std)
#             four = round(median+4*std)
#             five = round(median+5*std)
#             six = round(median+6*std)
#             seven = round(median+7*std)
#             eight = round(median+8*std)
#             nine =round(median+9*std)
#             ten = round(median+10*std)

#         elif std > 0:
#             print("std>0")

#             one = np.ceil(median+std)
#             two = np.ceil(one+std)
#             three = np.ceil(two+std)
#             four = np.ceil(three+std)
#             five = np.ceil(four+std)
#             six = np.ceil(five+std)
#             seven = np.ceil(six+std)
#             eight = np.ceil(seven+std)
#             nine =np.ceil(eight+std)
#             ten = np.ceil(nine+std)


#         elif std == 0:
#             print("std=0")
#             one = np.ceil(median+std+1)
#             two = np.ceil(one+std+1)
#             three = np.ceil(two+std+1)
#             four = np.ceil(three+std+1)
#             five = np.ceil(four+std+1)
#             six = np.ceil(five+std+1)
#             seven = np.ceil(six+std+1)
#             eight = np.ceil(seven+std+1)
#             nine =np.ceil(eight+std+1)
#             ten = np.ceil(nine+std+1)
            
#         else:
#             one = 'NULL'
#             two = 'NULL'
#             three = 'NULL'
#             four = 'NULL'
#             five = 'NULL'
#             six = 'NULL'
#             seven = 'NULL'
#             eight = 'NULL'
#             nine ='NULL'
#             ten = 'NULL'
            


#         l =[peer_group,rule_no,one,two,three,four,five,six,seven,eight,nine,ten]
#         peer_th.append(l)





In [44]:
thresh = pd.DataFrame(data = peer_th,columns = ['PEER_GROUP_ID','RULE_NO','ONE','TWO','THREE','FOUR','FIVE','SIX','SEVEN','EIGHT','NINE','TEN','MEAN','MEDIAN','MODE','STD','OUTLIER'])

In [45]:
thresh.fillna('NULL',inplace = True)

In [46]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [47]:
# INSERTING DATA IN THRESHOLD TABLE.
for ind,row in thresh.iterrows():
    cursor.execute("""insert into fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (THRESHOLD1_1,THRESHOLD1_2,THRESHOLD1_3,THRESHOLD1_4,THRESHOLD1_5,THRESHOLD1_6,THRESHOLD1_7,THRESHOLD1_8,THRESHOLD1_9,THRESHOLD1_10,MEAN,MEDIAN,MODE,STANDARD_DEVIATION,outlier,PEER_GROUP_ID,RULE_NO)values({},{},{},{},{},{},{},{},{},{},{},{},{},{},'{}','{}','{}' )""".format(row['ONE'],row['TWO'],row['THREE'],row['FOUR'],row['FIVE'],row['SIX'],row['SEVEN'],row['EIGHT'],row['NINE'],row['TEN'],row['MEAN'],row['MEDIAN'],row['MODE'],row['STD'],row['OUTLIER'],row['PEER_GROUP_ID'],row['RULE_NO']))
    conn.commit()

In [48]:
# cALCULATING THE RISK SCORE OF REPORT FOR EVERY RULE.
cursor.execute('''WITH CTE AS (
select
A.REPORT_ID,
A.RULE_NO,
coalesce((case 
	when c.THRESHOLD1_1 IS null then 1
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end ),1)RISK_SCORE,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES a 
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG b 
on a.REPORT_ID = b.REPORT_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.REPORT_TYPE and a.RULE_NO = c.RULE_NO
inner join FINCORE_DB.Fincore_Analytics.risk_lkp_rules rlr with (nolock)
on rlr.rule_no = a.rule_no
where rlr.RISK_SCORE_STATIC_flag = 0)
UPDATE  FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES SET RISK_SCORE_STATIC = CTE.RISK_SCORE
FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
INNER JOIN CTE
ON CTE.REPORT_ID = A.REPORT_ID AND CTE.RULE_NO = A.RULE_NO''')
conn.commit()

In [49]:
# POPULATING THE RISK_REPORT 
if len(report_rules_auto_high) == 0:
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
GETDATE () AS CREATE_DATE,round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE''')
    conn.commit()
    
elif len(report_rules_auto_high) == 1:
    
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
GETDATE () AS CREATE_DATE,
round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID <> {}
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high[0]))
    conn.commit()
    
    
else:
    report_rules_auto_high = tuple(report_rules_auto_high)
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
GETDATE () AS CREATE_DATE,round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID NOT IN {} 
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high))
    conn.commit()
    

In [50]:
conn.close()

In [51]:
print("First part completed")

First part completed
